<a href="https://colab.research.google.com/github/dongminkim0220/pytorch_tutorial/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

PyTorch Tutorial 15 - Transfer Learning

https://www.youtube.com/watch?v=K0lWSB2QoIQ&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4&index=16

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time, os, copy

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [7]:
data_transforms = {
    "train": transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean, std)
    ]),
    "val": transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean, std)
    ])
}

In [13]:
data_dir = "/content/drive/MyDrive/pytorch intro/hymenoptera_data"
sets = ['train', 'val']

In [14]:
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']
}

In [15]:
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, shuffle = True, num_workers = 0)
    for x in ['train', 'val']
}

In [16]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [17]:
class_names

['ants', 'bees']

## train

In [36]:
def train_model(model, criterion, optimizer, scheduler, num_epochs = 25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch: {epoch} / {num_epochs - 1}")
        print("-" * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best Val Acc {best_acc:4f}")

    model.load_state_dict(best_model_wts)
    return model

## load pre trained model

In [37]:
model = models.resnet18(pretrained = True)
for param in model.parameters():
    param.requires_grad = False

In [38]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

In [39]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [41]:
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [42]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs = 20)

Epoch: 0 / 19
----------
train Loss: 0.6959 | Acc: 0.5410
val Loss: 0.5433 | Acc: 0.7451

Epoch: 1 / 19
----------
train Loss: 0.5820 | Acc: 0.6844
val Loss: 0.3483 | Acc: 0.9412

Epoch: 2 / 19
----------
train Loss: 0.5394 | Acc: 0.7459
val Loss: 0.4476 | Acc: 0.8824

Epoch: 3 / 19
----------
train Loss: 0.4733 | Acc: 0.8115
val Loss: 0.2948 | Acc: 0.9412

Epoch: 4 / 19
----------
train Loss: 0.4257 | Acc: 0.8320
val Loss: 0.3587 | Acc: 0.9216

Epoch: 5 / 19
----------
train Loss: 0.4960 | Acc: 0.7500
val Loss: 0.2786 | Acc: 0.9804

Epoch: 6 / 19
----------
train Loss: 0.4299 | Acc: 0.8074
val Loss: 0.2259 | Acc: 0.9804

Epoch: 7 / 19
----------
train Loss: 0.4101 | Acc: 0.8443
val Loss: 0.2022 | Acc: 0.9804

Epoch: 8 / 19
----------
train Loss: 0.3991 | Acc: 0.8361
val Loss: 0.2895 | Acc: 0.9608

Epoch: 9 / 19
----------
train Loss: 0.4184 | Acc: 0.8320
val Loss: 0.2479 | Acc: 0.9804

Epoch: 10 / 19
----------
train Loss: 0.4208 | Acc: 0.8115
val Loss: 0.2657 | Acc: 0.9608

Epoch: 11